In [133]:
import numpy as np
import pandas as pd
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
import re
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer

In [138]:
train_df = pd.read_csv("train.csv")

In [7]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


In [8]:
train_df.isnull().sum()

id             0
keyword       61
location    2533
text           0
target         0
dtype: int64

In [96]:
train_df[:100]

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
95,137,accident,Charlotte,9 Mile backup on I-77 South...accident blockin...,1
96,138,accident,"Baton Rouge, LA",Has an accident changed your life? We will hel...,0
97,139,accident,"Hagerstown, MD",#BREAKING: there was a deadly motorcycle car a...,1
98,141,accident,"Gloucestershire , UK",@flowri were you marinading it or was it an ac...,0


In [27]:
print("Disaster total=",train_df[train_df["target"]==1]["target"].count())
print("non disaster total=",train_df[train_df["target"]==0]["target"].count())

Disaster total= 3271
non disaster total= 4342


In [31]:
keywords = train_df["keyword"].unique()    

In [32]:
keywords

array([nan, 'ablaze', 'accident', 'aftershock', 'airplane%20accident',
       'ambulance', 'annihilated', 'annihilation', 'apocalypse',
       'armageddon', 'army', 'arson', 'arsonist', 'attack', 'attacked',
       'avalanche', 'battle', 'bioterror', 'bioterrorism', 'blaze',
       'blazing', 'bleeding', 'blew%20up', 'blight', 'blizzard', 'blood',
       'bloody', 'blown%20up', 'body%20bag', 'body%20bagging',
       'body%20bags', 'bomb', 'bombed', 'bombing', 'bridge%20collapse',
       'buildings%20burning', 'buildings%20on%20fire', 'burned',
       'burning', 'burning%20buildings', 'bush%20fires', 'casualties',
       'casualty', 'catastrophe', 'catastrophic', 'chemical%20emergency',
       'cliff%20fall', 'collapse', 'collapsed', 'collide', 'collided',
       'collision', 'crash', 'crashed', 'crush', 'crushed', 'curfew',
       'cyclone', 'damage', 'danger', 'dead', 'death', 'deaths', 'debris',
       'deluge', 'deluged', 'demolish', 'demolished', 'demolition',
       'derail', 'der

In [34]:
keyword_text = train_df[train_df["keyword"].notna()][["keyword", "text"]]

In [35]:
keyword_text

,keyword,text
31,ablaze,@bbcmtd Wholesale Markets ablaze http://t.co/l...
32,ablaze,We always try to bring the heavy. #metal #RT h...
33,ablaze,#AFRICANBAZE: Breaking news:Nigeria flag set a...
34,ablaze,Crying out for more! Set me ablaze
35,ablaze,On plus side LOOK AT THE SKY LAST NIGHT IT WAS...
...,...,...
7578,wrecked,@jt_ruff23 @cameronhacker and I wrecked you both
7579,wrecked,Three days off from work and they've pretty mu...
7580,wrecked,#FX #forex #trading Cramer: Iger's 3 words tha...
7581,wrecked,@engineshed Great atmosphere at the British Li...


In [139]:
null_keyword_texts = train_df[train_df["keyword"].isnull()]["text"]

In [140]:
for i in range(len(train_df)):
    if str(train_df["keyword"].iloc[i]) == "nan":
        for word in keywords[1:]:
            if train_df["text"].loc[i].count(word)>=1:
                train_df["keyword"].loc[i] = word
            

/var/folders/zh/dvg4994d0_v8dpj_6tb0y44r0000gn/T/ipykernel_51693/4163110282.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df["keyword"].loc[i] = word


In [102]:
train_df.isnull().sum()

id             0
keyword        0
location    2533
text           0
target         0
dtype: int64

In [141]:
for i in range(len(train_df)):
    if str(train_df["keyword"].iloc[i]) == "nan":
        train_df["keyword"].loc[i] = "blarg"

/var/folders/zh/dvg4994d0_v8dpj_6tb0y44r0000gn/T/ipykernel_51693/3430923347.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df["keyword"].loc[i] = "blarg"


In [142]:
train_df["keyword"].value_counts()

fatalities               45
armageddon               42
fire                     42
deluge                   42
earthquake               41
                         ..
forest%20fire            19
epicentre                12
threat                   12
inundation               10
radiation%20emergency     9
Name: keyword, Length: 222, dtype: int64

In [143]:
for i in range(len(train_df)):
    if train_df["keyword"].loc[i][-1] == "s" and train_df["keyword"].loc[i][-2] not in ["s","a","e","i","o","u"]:
        train_df["keyword"].loc[i] = train_df["keyword"].loc[i][:-1]

/var/folders/zh/dvg4994d0_v8dpj_6tb0y44r0000gn/T/ipykernel_51693/2740022522.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df["keyword"].loc[i] = train_df["keyword"].loc[i][:-1]


In [144]:
train_df["keyword"].value_counts()

weapon                   78
death                    74
body%20bag               74
flood                    71
siren                    69
                         ..
forest%20fire            19
threat                   12
epicentre                12
inundation               10
radiation%20emergency     9
Name: keyword, Length: 217, dtype: int64

In [80]:
chars = ["!", "#", ",", ".", ":", ";", "?", "-", "~", ")", "(", "=", ">", "<", "/"]

In [106]:
text_no_special = []
for i in range(len(train_df)):
    text = train_df["text"].loc[i]
    text = text.lower()

    #Removing the @user
    text = re.sub(r'@[^\s]+', '', text)

    #Removing the links
    text = re.sub(r'http:\S+', '', text)
    text = re.sub(r'https:\S+', '', text)

    #Removing non ASCII characters
    text = re.sub(r'[^\x00-\x7F]+', '', text)
    for letter in text:
        if letter in chars:
            c+=1
            text = text.replace(letter,"")
            
    #if c>=1:
    #    print(text)
    #    print(train_df["text"].loc[i])
    text_no_special.append(text)

    

In [145]:
train_df["text_no_special"] = text_no_special

In [119]:
def char_in_text_plot(char,df):
    count_fine, count_disaster=0,0
    for i in range(len(df["text"])):
        text = df["text"].loc[i]
        if char in text:
            if df["target"].loc[i] ==0:
                count_fine+=1
            else:
                count_disaster +=1
    if (count_fine+count_disaster) > 1:
        return count_fine/(count_fine+count_disaster),count_disaster/(count_fine+count_disaster),count_fine+count_disaster
    else:
        return 0,0,0

In [120]:
for char in chars:
    fine, disaster, total = char_in_text_plot(char,train_df)
    print(char,":","fine= ",fine,"disaster= ",disaster,"total=",total)
    print("**************************")

! : fine=  0.7273991655076495 disaster=  0.2726008344923505 total= 719
**************************
# : fine=  0.5031232254400908 disaster=  0.49687677455990914 total= 1761
**************************
, : fine=  0.0 disaster=  1.0 total= 2
**************************
. : fine=  0.5125044029587883 disaster=  0.4874955970412117 total= 5678
**************************
: : fine=  0.4613793103448276 disaster=  0.5386206896551724 total= 4350
**************************
; : fine=  0.5698198198198198 disaster=  0.43018018018018017 total= 444
**************************
? : fine=  0.7197696737044146 disaster=  0.2802303262955854 total= 1042
**************************
- : fine=  0.4694035631293571 disaster=  0.530596436870643 total= 1291
**************************
~ : fine=  0.4444444444444444 disaster=  0.5555555555555556 total= 27
**************************
) : fine=  0.6151603498542274 disaster=  0.3848396501457726 total= 343
**************************
( : fine=  0.6024464831804281 disaster=  0.3975

In [121]:
sig_chars = ["?","!"]

In [126]:
sig_chars_enc = []
for text in train_df["text"]:
    c = 0
    for char in sig_chars:
        c+=text.count(char)
    sig_chars_enc.append(c)
            

In [146]:
train_df["sig_chars"] = sig_chars_enc

In [147]:
train_df

,id,keyword,location,text,target,text_no_special,sig_chars
0,1,earthquake,NaN,Our Deeds are the Reason of this #earthquake M...,1,our deeds are the reason of this earthquake ma...,0
1,4,fire,NaN,Forest fire near La Ronge Sask. Canada,1,forest fire near la ronge sask canada,0
2,5,evacuation,NaN,All residents asked to 'shelter in place' are ...,1,all residents asked to 'shelter in place' are ...,0
3,6,wildfire,NaN,"13,000 people receive #wildfires evacuation or...",1,13000 people receive wildfires evacuation orde...,0
4,7,wildfire,NaN,Just got sent this photo from Ruby #Alaska as ...,1,just got sent this photo from ruby alaska as s...,0
...,...,...,...,...,...,...,...
7608,10869,collapse,NaN,Two giant cranes holding a bridge collapse int...,1,two giant cranes holding a bridge collapse int...,0
7609,10870,fire,NaN,@aria_ahrary @TheTawniest The out of control w...,1,the out of control wild fires in california ...,0
7610,10871,blarg,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1,m194 [0104 utc]5km s of volcano hawaii,1
7611,10872,threat,NaN,Police investigating after an e-bike collided ...,1,police investigating after an ebike collided w...,0


In [148]:
train_df["location"] = train_df["location"].fillna(value=-1)

In [154]:
drop = ["id","location","text","target","text_no_special"]
count = ["text_no_special"]

In [129]:
count_vectorizer = feature_extraction.text.CountVectorizer()

In [163]:
text_trans=count_vectorizer.fit_transform(train_df["text_no_special"]).todense()